#### Problem statement:
#### 1.Detecting fraud transactions is of great importance for any credit card company.
#### 2.We are tasked by a well-known company to detect potential frauds so that customers are not charged for items that they did not purchase.

#### 3.So the goal is to build a classifier that tells if a transaction is a fraud or not.####

#### Import the library #### 

In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE

#### Data processing ####

In [53]:
dataset = pd.read_csv("D:\\Kaggle Datasets\\creditcard.csv")
dataset.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [54]:
dataset.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.165980e-15,3.416908e-16,-1.373150e-15,2.086869e-15,9.604066e-16,1.490107e-15,-5.556467e-16,1.177556e-16,-2.406455e-15,...,1.656562e-16,-3.444850e-16,2.578648e-16,4.471968e-15,5.340915e-16,1.687098e-15,-3.666453e-16,-1.220404e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


#### We can drop the ['Amount','Time'] columns as they would not help in model building ####

In [55]:
df = dataset.drop(columns = ['Amount','Time'], axis = 1)
X = df.iloc[: ,:-1].values
y = df.iloc[: , -1].values

#### Split the data into train and test datasets ####

In [56]:
from sklearn.model_selection import train_test_split
X_train , X_test, y_train, y_test = train_test_split(X , y, test_size = 0.3, random_state = 42)

### DNN Model building###


#### We will build a neural network using Sequential model in keras ####

In [57]:
ann_model = tf.keras.models.Sequential()
ann_model.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))
ann_model.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))
ann_model.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

### DNN Model Evaluation ###

#### Let's compile and train the model ####

In [58]:
ann_model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['accuracy'])
ann_model.fit(X_train, y_train, batch_size = 32, epochs = 10)

Train on 199364 samples
Epoch 1/10
199364/199364 [==============================] - 17s 88us/sample - loss: 0.0308 - accuracy: 0.9909
Epoch 2/10
199364/199364 [==============================] - 11s 56us/sample - loss: 0.0037 - accuracy: 0.9994
Epoch 3/10
199364/199364 [==============================] - 11s 53us/sample - loss: 0.0035 - accuracy: 0.9994
Epoch 4/10
199364/199364 [==============================] - 10s 51us/sample - loss: 0.0033 - accuracy: 0.9994
Epoch 5/10
199364/199364 [==============================] - 11s 53us/sample - loss: 0.0031 - accuracy: 0.9994
Epoch 6/10
199364/199364 [==============================] - 11s 53us/sample - loss: 0.0030 - accuracy: 0.9994
Epoch 7/10
199364/199364 [==============================] - 10s 52us/sample - loss: 0.0030 - accuracy: 0.9994
Epoch 8/10
199364/199364 [==============================] - 10s 50us/sample - loss: 0.0030 - accuracy: 0.9994
Epoch 9/10
199364/199364 [==============================] - 10s 49us/sample - loss: 0.0029 - acc

#### Let's evaluate the model ####

In [59]:
score = ann_model.evaluate(X_test, y_test)
print(score)

85443/85443 [==============================] - 3s 34us/sample - loss: 0.0027 - accuracy: 0.9994
[0.0026729599015297, 0.9994148]


#### The model is found to have 99.94% accuracy ####

#### If 95% of the dataset is negative(Non-frauds) the network will cleverly predict all to be negative leading to 95% accuracy.However, for fraud detection, detecting positive is very important.Therefore, we need better metrics. ####

#### Predicting the Test set Results ####

In [60]:
y_pred = ann_model.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 1]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


#### Making the confusion matrix #### 

In [61]:
cm = confusion_matrix(y_test,y_pred)
print(cm)

[[85283    24]
 [   26   110]]


In [62]:
print(precision_score(y_test, y_pred.round()))
print(recall_score(y_test, y_pred.round()))
print(f1_score(y_test, y_pred.round()))

0.8208955223880597
0.8088235294117647
0.8148148148148148


#### The confusion matrix shows a Precision of 82%, Recall of 80.8% and F1 Score of 81.4 %. About 20% of frauds are misclassified as non-frauds , leading to extra payments for customers, though the accuracy is 99.94%. So there is enough space to improve the DNN model.####

#### Let's build a decision tree ####

In [63]:
from sklearn.tree import DecisionTreeClassifier
decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(X_train, y_train)
y_pred = decision_tree_model.predict(X_test)

In [64]:
cm = confusion_matrix(y_test,y_pred)
print(cm)

[[85267    40]
 [   29   107]]


In [65]:
print(precision_score(y_test, y_pred.round()))
print(recall_score(y_test, y_pred.round()))
print(f1_score(y_test, y_pred.round()))

0.7278911564625851
0.7867647058823529
0.7561837455830389


#### The Decision Tree gives a Precision of 72.7 %, Recall of 78.6 %, F1 Score of  75.6% worse than DNN model.####

#### Let's build a Random Forest ####

In [66]:
from sklearn.ensemble import RandomForestClassifier
random_forest_model = RandomForestClassifier(n_estimators = 100)
random_forest_model.fit(X_train, y_train)
y_pred = random_forest_model.predict(X_test)

In [67]:
def conf_mat(model):
    cm = confusion_matrix(y_test, y_pred)
    print(cm)

In [68]:
conf_mat(random_forest_model)

[[85297    10]
 [   26   110]]


In [69]:
print(precision_score(y_test, y_pred.round()))
print(recall_score(y_test, y_pred.round()))
print(f1_score(y_test, y_pred.round()))

0.9166666666666666
0.8088235294117647
0.859375


#### The Random Forest gives a Precision of 91.6%, Recall of 80.8% and F1 Score of  85.9% better than DNN and Decision Tree ####

#### We have a class imbalance issue out here.The model is more sensitive to detect majority class than minority class.####

### Undersampling ###

In [70]:
fraud_ind = np.array(df[df.Class == 1].index)
num_frauds = len(fraud_ind)
print(num_frauds)
normal_ind = np.array(df[df.Class == 0].index)
num_normal = len(normal_ind)
print(num_normal)

492
284315


In [71]:
normal_ind = df[df.Class == 0].index
random_normal_ind = np.random.choice(normal_ind, num_frauds, replace = False)
random_normal_ind = np.array(random_normal_ind)

In [72]:
under_sample_ind = np.concatenate( [fraud_ind, random_normal_ind])

In [73]:
print(len(under_sample_ind))

984


In [74]:
under_sample_data = df.iloc[under_sample_ind, :]

In [75]:
X_undersample = under_sample_data.iloc[:, under_sample_data.columns != 'Class']
y_undersample = under_sample_data.iloc[:, under_sample_data.columns == 'Class']

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X_undersample, y_undersample, test_size = 0.3)

In [77]:
ann_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann_model.fit(X_train,y_train, batch_size = 32, epochs = 10)

Train on 688 samples
Epoch 1/10
688/688 [==============================] - 1s 2ms/sample - loss: 0.4953 - accuracy: 0.9099
Epoch 2/10
688/688 [==============================] - 0s 57us/sample - loss: 0.3884 - accuracy: 0.9230
Epoch 3/10
688/688 [==============================] - 0s 57us/sample - loss: 0.3225 - accuracy: 0.9317
Epoch 4/10
688/688 [==============================] - 0s 64us/sample - loss: 0.2792 - accuracy: 0.9375
Epoch 5/10
688/688 [==============================] - 0s 59us/sample - loss: 0.2463 - accuracy: 0.9419
Epoch 6/10
688/688 [==============================] - 0s 59us/sample - loss: 0.2211 - accuracy: 0.9477
Epoch 7/10
688/688 [==============================] - 0s 68us/sample - loss: 0.2014 - accuracy: 0.9491
Epoch 8/10
688/688 [==============================] - 0s 61us/sample - loss: 0.1850 - accuracy: 0.9462
Epoch 9/10
688/688 [==============================] - 0s 59us/sample - loss: 0.1715 - accuracy: 0.9462
Epoch 10/10
688/688 [==============================] 

In [78]:
y_pred = ann_model.predict(X_test)

In [79]:
cm = confusion_matrix(y_test, y_pred.round())
print(cm)

[[141   2]
 [ 16 137]]


In [80]:
print(accuracy_score(y_test, y_pred.round()))
print(precision_score(y_test, y_pred.round()))
print(recall_score(y_test, y_pred.round()))
print(f1_score(y_test, y_pred.round()))

0.9391891891891891
0.9856115107913669
0.8954248366013072
0.9383561643835616


#### Undersampling gives a Precision of 98.5%, Recall of 89.5% and F1 Score of 93.8% .Much better than DNN without undersampling. ####

### SMOTE ### 

In [81]:
X_resample, y_resample = SMOTE().fit_resample(X , y)

In [82]:
X_resample.shape

(568630, 28)

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X_resample, y_resample, test_size = 0.3)

In [84]:
ann_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann_model.fit(X_train, y_train, batch_size = 32, epochs = 10)

Train on 398041 samples
Epoch 1/10
398041/398041 [==============================] - 21s 52us/sample - loss: 0.0616 - accuracy: 0.9742
Epoch 2/10
398041/398041 [==============================] - 20s 51us/sample - loss: 0.0369 - accuracy: 0.9868
Epoch 3/10
398041/398041 [==============================] - 20s 50us/sample - loss: 0.0300 - accuracy: 0.9903
Epoch 4/10
398041/398041 [==============================] - 20s 50us/sample - loss: 0.0267 - accuracy: 0.9916
Epoch 5/10
398041/398041 [==============================] - 20s 50us/sample - loss: 0.0246 - accuracy: 0.9925
Epoch 6/10
398041/398041 [==============================] - 20s 50us/sample - loss: 0.0240 - accuracy: 0.9928
Epoch 7/10
398041/398041 [==============================] - 20s 50us/sample - loss: 0.0234 - accuracy: 0.9929
Epoch 8/10
398041/398041 [==============================] - 20s 50us/sample - loss: 0.0229 - accuracy: 0.9932
Epoch 9/10
398041/398041 [==============================] - 20s 51us/sample - loss: 0.0225 - acc

In [85]:
y_pred = ann_model.predict(X_test)

In [86]:
cm = confusion_matrix(y_test, y_pred.round())
print(cm)

[[84317   914]
 [   98 85260]]


In [87]:
print(accuracy_score(y_test, y_pred.round()))
print(precision_score(y_test, y_pred.round()))
print(recall_score(y_test, y_pred.round()))
print(f1_score(y_test, y_pred.round()))

0.9940676128003564
0.9893935525796644
0.9988518943742825
0.9941002261968613


### We created 5 models, DNN, Decision Tree, Random Forest, DNN with undersampling and DNN with SMOTE.DNN with SMOTE performs the best.###